<a href="https://colab.research.google.com/github/FrankPerera04/DSPL_Individual_CW/blob/main/Developing%20Streamlit/Setting_up_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions for Usage**
1. Install in packages
2. Write the App
3. Start streamlit and tunnel
4. Get the ipv4 code
5. Run and get the Tunnel link
6. Enter the ipv4 code into the tunnel website to access streamlit

In [ ]:
!pip install streamlit -q
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [ ]:
# Write the app
%%writefile app.py
import streamlit as st

# Set page config IMMEDIATELY after importing streamlit
st.set_page_config(page_title="Accommodation Dashboard", layout="wide")

import pandas as pd
import plotly.express as px

# Load data from Google Drive path
@st.cache_data
def load_data():
    # Change this to your Google Drive path if needed
    return pd.read_csv("https://raw.githubusercontent.com/FrankPerera04/DSPL_Individual_CW/refs/heads/main/processed_accommodation_data%20(1).csv")

df = load_data()

# Dashboard title
st.title("🏢 Accommodation Insights Dashboard")

# Overview cards
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Records", len(df))
col2.metric("Unique Districts", df["District"].nunique())
col3.metric("Accommodation Types", df["Type"].nunique())
col4.metric("Total Rooms", int(df["Rooms"].sum()))

# Sidebar filters
with st.sidebar:
    st.header("🔍 Filters")
    selected_districts = st.multiselect(
        "Select District(s)",
        df["District"].unique(),
        default=df["District"].unique()
    )
    selected_types = st.multiselect(
        "Select Accommodation Type(s)",
        df["Type"].unique(),
        default=df["Type"].unique()
    )

# Apply filters
filtered_df = df[
    (df["District"].isin(selected_districts)) &
    (df["Type"].isin(selected_types))
]






# Accommodation Metrics by District
st.subheader("Accommodation Metrics by District")

# Grouping data
metrics_df = filtered_df.groupby("District").agg(
    Total_Accommodations=("Name", "count"),
    Total_Rooms=("Rooms", "sum"),
    Average_Rooms_Per_Property=("Rooms", "mean")
).reset_index()

# Round average to 1 decimal
metrics_df["Average_Rooms_Per_Property"] = metrics_df["Average_Rooms_Per_Property"].round(1)



# 1. Total Number of Accommodations per District
st.subheader("1 - Total Number of Accommodations per District")

# Sort the dataframe by Total_Accommodations descending
metrics_df_sorted = metrics_df.sort_values(by="Total_Accommodations", ascending=False)

fig_acc = px.bar(
    metrics_df_sorted,
    x="District",
    y="Total_Accommodations",
    color="Total_Accommodations",
    text="Total_Accommodations",
    template="plotly_dark"
)

fig_acc.update_layout(
    xaxis_title="District",
    yaxis_title="Number of Accommodations",
    xaxis_tickangle=-45
)

st.plotly_chart(fig_acc, use_container_width=True)



# 2. Total Number of Rooms per District
st.subheader("2 - Total Number of Rooms per District")

# Sort the dataframe by Total_Rooms descending
metrics_df_rooms_sorted = metrics_df.sort_values(by="Total_Rooms", ascending=False)

fig_rooms = px.bar(
    metrics_df_rooms_sorted,
    x="District",
    y="Total_Rooms",
    color="Total_Rooms",
    text="Total_Rooms",
    template="plotly_dark"
)
fig_rooms.update_layout(xaxis_title="District", yaxis_title="Number of Rooms", xaxis_tickangle=-45)
st.plotly_chart(fig_rooms, use_container_width=True)



# 3. Average Size of Accommodations (Rooms per Property)
st.subheader("3 - Average Size of Accommodations (Rooms per Property)")

# Sort the dataframe by Average_Rooms_Per_Property descending
metrics_df_avg_sorted = metrics_df.sort_values(by="Average_Rooms_Per_Property", ascending=False)

fig_avg = px.bar(
    metrics_df_avg_sorted,
    x="District",
    y="Average_Rooms_Per_Property",
    color="Average_Rooms_Per_Property",
    text="Average_Rooms_Per_Property",
    template="plotly_dark"
)
fig_avg.update_layout(xaxis_title="District", yaxis_title="Average Rooms per Property", xaxis_tickangle=-45)
st.plotly_chart(fig_avg, use_container_width=True)






#Geographic Gaps
# Map of Accommodations + Highlight Low Accommodation Districts
st.subheader("🗺️ Geographic Gaps in Accommodation Availability")

# Prepare counts
district_counts = filtered_df.groupby("District").agg(
    Total_Accommodations=("Name", "count")
).reset_index()

# Merge counts back with filtered data
map_df = pd.merge(filtered_df, district_counts, on="District", how="left")

# Plotting the accommodations on map
st.map(
    map_df[["Latitude", "Logitiute"]].rename(columns={"Logitiute": "longitude", "Latitude": "latitude"})
)

# Highlight districts with very few accommodations
low_accommodation_threshold = 5  # You can adjust this threshold
low_acc_districts = district_counts[district_counts["Total_Accommodations"] <= low_accommodation_threshold]

if not low_acc_districts.empty:
    st.warning("⚠️ Districts with very few accommodations (<= 5):")
    st.dataframe(low_acc_districts, use_container_width=True)
else:
    st.success("✅ All districts have sufficient accommodation coverage.")







# Type Distribution
# Accommodation Type Distribution
st.subheader("🏘️ Accommodation Type Distribution")

# Group data by Type
type_distribution = filtered_df["Type"].value_counts().reset_index()
type_distribution.columns = ["Accommodation Type", "Count"]

# Pie Chart: Distribution of Accommodation Types
fig_type = px.pie(
    type_distribution,
    names="Accommodation Type",
    values="Count",
    title="Accommodation Types Share",
    hole=0.4,
    template="plotly_dark"
)
st.plotly_chart(fig_type, use_container_width=True)

# Optional: Show data table
st.dataframe(type_distribution, use_container_width=True)











# Action Signals
# Action Signals: Highlighting Low Accommodation Districts
st.subheader("🚨 Action Signals: Districts Needing Accommodation Improvement")

# Set a threshold for "low accommodation" (example: less than or equal to 5)
low_threshold = 5

# Group by district
action_df = filtered_df.groupby("District").agg(
    Total_Accommodations=("Name", "count"),
    Total_Rooms=("Rooms", "sum")
).reset_index()

# Filter districts below threshold
low_accommodation_districts = action_df[action_df["Total_Accommodations"] <= low_threshold]

# Show results
if not low_accommodation_districts.empty:
    st.warning(f"⚠️ {len(low_accommodation_districts)} District(s) have <= {low_threshold} accommodations. Consider investing here!")
    st.dataframe(low_accommodation_districts, use_container_width=True)

    # Optional: Visualize with bar chart
    fig_low = px.bar(
        low_accommodation_districts,
        x="District",
        y="Total_Accommodations",
        color="Total_Accommodations",
        text="Total_Accommodations",
        title="Districts with Low Number of Accommodations",
        template="plotly_dark"
    )
    fig_low.update_layout(xaxis_title="District", yaxis_title="Total Accommodations", xaxis_tickangle=-45)
    st.plotly_chart(fig_low, use_container_width=True)

else:
    st.success("✅ All districts have sufficient number of accommodations!")


# Data table: Show filtered data
st.subheader("📋 Filtered Accommodation Data")
st.dataframe(filtered_df, use_container_width=True)



Writing app.py


In [ ]:
# Start Streamlit and tunnel in the background
import threading
import time

def run_app():
    get_ipython().system('streamlit run app.py &')

threading.Thread(target=run_app).start()
time.sleep(10)  # wait for Streamlit to start




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.57.220.32:8501



In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.57.220.32


In [ ]:
# Create the tunnel
get_ipython().system('npx localtunnel --port 8501')

⠙⠹⠸⠼⠴your url is: https://quiet-feet-like.loca.lt
────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/exec_code.py:  
  121 in exec_func_with_error_handling                                                  
                                                                                        
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/script_runner  
  .py:640 in code_to_exec                                                               
                                                                                        
  /content/app.py:15 in <module>                                                        
                                                                                        
     12 │   # Change this to your Google Drive path if needed                           
     13 │   return pd.read_csv("/content/drive/MyDrive/proce